## File to optimize routes simultaneously

In [1]:
using CSV, DataFrames, JuMP, Gurobi

[ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]


In [17]:
data = CSV.read("data_route1.csv", DataFrame)


x = data[:, 5]
y = data[:, 6]

# distance matrix from x and y
dist = zeros(n, n)
for i in 1:length(x)
    for j in 1:length(x)
        dist[i, j] = sqrt((x[i] - x[j])^2 + (y[i] - y[j])^2)
    end
end
dist = Matrix(dist);
dist

24×24 Matrix{Float64}:
 0.0         0.0038998   0.00581405  …  0.0484533   0.0512928   0.0538272
 0.0038998   0.0         0.00203093     0.0496005   0.0525027   0.0551407
 0.00581405  0.00203093  0.0            0.0494743   0.0524038   0.0550928
 0.00724959  0.0033498   0.00170407     0.0507976   0.0537395   0.0564529
 0.0103872   0.00654777  0.00506829     0.0533614   0.0563238   0.0590825
 0.0135484   0.00964908  0.00785573  …  0.0537131   0.0566979   0.0595205
 0.0126507   0.00878222  0.00684401     0.0519301   0.0549144   0.0577353
 0.0117351   0.00810541  0.00607617     0.049308    0.0522915   0.0551103
 0.0114781   0.00883208  0.00719029     0.0448824   0.0478644   0.0506795
 0.0123063   0.0106184   0.00942731     0.0414495   0.0444292   0.0472381
 0.0127855   0.0114281   0.0103832   …  0.0401999   0.0431783   0.0459835
 0.015601    0.0149162   0.014117       0.0362145   0.0391959   0.0420118
 0.0183403   0.0178466   0.0170844      0.0334512   0.0364402   0.039284
 0.0212362   0.0

In [15]:
data = CSV.read("data_route1_time_periods.csv", DataFrame)

data_matrix = []
for i in 1:11
    if i >= 10
        time_period = "time_period_" * string(i)
    else
        time_period = "time_period_0" * string(i)
    end
    # subset of data when time_period_id is time_period_i
    data_subset = data[data[!, "time_period_id"] .== time_period, :]
    data_subset = Matrix(data_subset)
    push!(data_matrix, data_subset)
end

x = data_matrix[1][:, 5]
y = data_matrix[1][:, 6]
n = length(x)

p = 11

# create on_data matrix with one column for each time period
on_data_matrix = zeros(n, 11)
for i in 1:11
    on_data_matrix[:, i] = data_matrix[i][:, 2]
end

# create off_data matrix with one column for each time period
off_data_matrix = zeros(n, 11)
for i in 1:11
    off_data_matrix[:, i] = data_matrix[i][:, 3]
end

load_data_matrix = zeros(n, 11)
for i in 1:11
    load_data_matrix[:, i] = data_matrix[i][:, 4]
end

In [6]:
function bus_model_time_periods(lambda)

    M = 1000

    model = Model(Gurobi.Optimizer)
    set_attribute(model, "OutputFlag", 0)

    @variable(model, z[1:n, 1:p], Bin)
    @variable(model, load[1:n, 1:p] >= 0)
    @variable(model, off[1:n, 1:p] >= 0)
    @variable(model, y[1:p, 1:4], Bin)
    @variable(model, s[1:p, 1:4], Bin) # if s_tj = 1, then y_tj + y_t+1,j + y_t_2,j + y_t+3,j = 1

    @constraint(model, [i in 1:n, t in 1:p], load[i, t] <= 40)
    # load at bus stop j = load at bus stop j-1 + z_j * (number_go_on_at_stop_j - number_go_off_at_j-1)
    @constraint(model, [t in 1:p], load[1, t] == load_data_matrix[1, t])
    @constraint(model, [t in 1:p], load[n, t] <= 1)
    @constraint(model, [i in 2:n, t in 1:p], load[i, t] == load[i-1, t] + z[i, t] * (on_data_matrix[i, t] - off[i-1, t]))
    # If a stop is removed, people go go off at the next available stop off_j = off_data_j + (1-z_{j-1})*off_j-1
    @constraint(model, [t in 1:p], off[1, t] == off_data_matrix[1, t])
    @constraint(model, [i in 2:n, t in 1:p], off[i, t] == off_data_matrix[i, t] + (1-z[i-1, t]) * off[i-1, t])
    # bus has to stop at first and last stop
    @constraint(model, [t in 1:p], z[1, t] == 1)
    @constraint(model, [t in 1:p], z[n, t] == 1)
    # time periods constraints: group time periods into 4 groups (must be consecutive) because we dont want to change stops too often
    @constraint(model, sum(y[t, 1] for t in 1:p) == 3)
    @constraint(model, sum(y[t, 2] for t in 1:p) == 3)
    @constraint(model, sum(y[t, 3] for t in 1:p) == 3)
    @constraint(model, sum(y[t, 4] for t in 1:p) == 2)
    @constraint(model, [t in 1:p], sum(y[t, j] for j in 1:4) == 1)  # each time period is in exactly one group
    # @constraint(model, [t in 1:p, j in 1:4], s[t, j] <= y[t, j])
    # @constraint(model, [t in 1:p, j in 1:4], s[t, j] <= y[t%11 + 1, j])
    # @constraint(model, [t in 1:p, j in 1:4], s[t, j] <= y[(t+1)%11 + 1, j])
    @constraint(model, [t in 1:p, j in 1:3], 3*s[t, j] <= y[t, j] + y[t%11 + 1, j] + y[(t+1)%11 + 1, j])
    @constraint(model, [t in 1:p], 2*s[t, 4] <= y[t, 4] + y[t%11 + 1, 4])
    @constraint(model, [j in 1:4], sum(s[t, j] for t in 1:p) == 1)
    @constraint(model, [i in 1:n, t in 1:p, k in 1:p, j in 1:4], z[i, t] <= z[i, k] + M*(2 - y[t, j] - y[k, j])) # if time t and k are in the same group, then z[i, t] <= z[i, k]
    @constraint(model, [i in 1:n, t in 1:p, k in 1:p, j in 1:4], z[i, t] >= z[i, k] - M*(2 - y[t, j] - y[k, j])) # if time t and k are in the same group, then z[i, t] >= z[i, k]

    # CO2 consumption is sum(load[i] * distance to go from i to i+1) + (penalty if bus stops at stop i)
    @objective(model, Max, lambda * sum(z[i, t] * on_data_matrix[i, t] for i in 1:n for t in 1:p) - (1 - lambda) * (sum(load[i, t] * dist[i, i+1] for i in 1:n-1 for t in 1:p) + sum(z[i, t] for i in 1:n for t in 1:p)))

    optimize!(model);

    return z, load, off, y, s, objective_value(model)
end

bus_model_time_periods (generic function with 1 method)

In [18]:
data_matrix[1]

24×9 Matrix{Any}:
    64  10.375   0.0     …  "NUBIAN STATION"
     1   0.15    0.0125     "WASHINGTON ST OPP RUGGLES ST"
     2   0.175   0.6875     "WASHINGTON ST @ MELNEA CASS BLVD"
     6   0.0125  0.0        "MELNEA CASS BLVD @ HARRISON AVE"
 10003   0.2375  0.15       "ALBANY ST OPP RANDALL ST"
    57   1.3125  1.5125  …  "MASSACHUSETTS AVE @ ALBANY ST"
    58   1.0875  0.8875     "MASSACHUSETTS AVE @ HARRISON AVE"
 10590   1.2875  0.1625     "MASSACHUSETTS AVE @ WASHINGTON ST"
    87   0.8375  0.175      "MASSACHUSETTS AVE @ TREMONT ST"
   188   1.575   0.975      "MASSACHUSETTS AVE @ MASSACHUSETTS AVE STATION"
    89   0.5375  0.6625  …  "MASSACHUSETTS AVE @ ST BOTOLPH ST"
    91   0.45    0.35       "MASSACHUSETTS AVE @ CLEARWAY ST"
    93   1.1625  2.3875     "MASSACHUSETTS AVE @ NEWBURY ST"
    95   0.2875  0.125      "MASSACHUSETTS AVE @ BEACON ST"
    97   0.0625  1.5125     "77 MASSACHUSETTS AVE"
    99   0.3125  1.2     …  "MASSACHUSETTS AVE @ ALBANY ST 2"
   101   0.45 

In [19]:
z, load, off, y, s, _ = bus_model_time_periods(0.5);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


In [21]:
value.(z)

24×11 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0